In [1]:
# Create comprehensive dashboard dataset
import pandas as pd

# Load main data
df = pd.read_csv('../data/ecommerce_funnel_data.csv')

# Add month for time analysis
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month'] = df['timestamp'].dt.month_name()
df['quarter'] = 'Q' + df['timestamp'].dt.quarter.astype(str)

# Create shipping tier
df['shipping_tier'] = pd.cut(
    df['shipping_cost'],
    bins=[-0.01, 0.01, 10, 15, 100],
    labels=['Free', 'Low ($1-10)', 'Medium ($11-15)', 'High (>$15)']
)

# Create cart value tier
df['cart_value_tier'] = pd.cut(
    df['cart_value'],
    bins=[0, 100, 200, 300, 1000],
    labels=['<$100', '$100-200', '$200-300', '>$300']
)

# Calculate funnel stage
def get_funnel_stage(row):
    if row['completed_purchase']:
        return '5. Purchased'
    elif row['reached_checkout']:
        return '4. Checkout'
    elif row['added_to_cart']:
        return '3. Cart'
    elif row['viewed_product']:
        return '2. Viewed'
    else:
        return '1. Browsed'

df['funnel_stage'] = df.apply(get_funnel_stage, axis=1)

# Add abandonment flag for cart users
df['abandoned_cart'] = ((df['added_to_cart'] == True) & 
                        (df['reached_checkout'] == False)).astype(int)

# Revenue (only for completed purchases)
df['revenue'] = df['cart_value'] * df['completed_purchase']

# Save consolidated file
df.to_csv('../data/tableau_dashboard_data.csv', index=False)

print("✅ Tableau dashboard data created!")
print(f"📊 File: ../data/tableau_dashboard_data.csv")
print(f"📊 Rows: {len(df):,}")
print(f"📊 Columns: {len(df.columns)}")
print(f"\nColumns included:")
for col in df.columns:
    print(f"   • {col}")

✅ Tableau dashboard data created!
📊 File: ../data/tableau_dashboard_data.csv
📊 Rows: 100,000
📊 Columns: 21

Columns included:
   • session_id
   • timestamp
   • device
   • traffic_source
   • products_viewed
   • time_on_page_seconds
   • cart_value
   • shipping_cost
   • has_discount
   • browsed
   • viewed_product
   • added_to_cart
   • reached_checkout
   • completed_purchase
   • month
   • quarter
   • shipping_tier
   • cart_value_tier
   • funnel_stage
   • abandoned_cart
   • revenue
